In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd

# For reproducibility
np.random.seed(42)
tf.random.set_seed(42)


In [11]:
# Load the JSON and Excel datasets
x_df = pd.read_json('../DataSet/assignment1New.json')
# y_df = pd.read_excel('../DataBook/crafAssign/Assignment1_Data_Analyst.xlsx')

# # Display the first few rows of each DataFrame
# print("x_df head:\n", x_df.head())
# print("y_df head:\n", y_df.head())


FileNotFoundError: File ../DataSet/individ/assignment1New.json does not exist

In [8]:
# Load the Excel dataset from the specified path
excel_path = '../DataBook/crafAssign/Assignment1_Data_Analyst.xlsx'
try:
    y_df = pd.read_excel(excel_path)
    print("Data loaded successfully.")
    print("First few rows:")
    print(y_df.head())
except FileNotFoundError:
    print(f"File '{excel_path}' not found. Please check the file path.")
except Exception as e:
    print(f"Error loading Excel file: {str(e)}")



File '../DataBook/crafAssign/Assignment1_Data_Analyst.xlsx' not found. Please check the file path.
